In [38]:
import torch
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import sys
sys.path.append("../../")
from TCN.mnist_pixel.utils import data_generator
from TCN.mnist_pixel.model import TCN
import numpy as np
import argparse

In [2]:
#not needed
def customOneHotEncoder(data):
    dataAdjust = data.ljust(200,'0')[:200] # padding if not of length and adjusting the data lenght to get a 200x39 input matrix
    # define universe of possible input values
    alphabet = '0123456789abcdefghijklmnopqrstuvwxyz,._-/+'
    # define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(alphabet))
    int_to_char = dict((i, c) for i, c in enumerate(alphabet))
    # integer encode input data
    integer_encoded = [char_to_int[char] for char in dataAdjust]
    #print(integer_encoded)
    # one hot encode
    onehot_encoded = list()
    for i, value in enumerate(integer_encoded):
        letter = [0 for _ in range(len(alphabet))]
        letter[value] = 1
        onehot_encoded.append(letter)
    #print(onehot_encoded) # the real encoding
    return onehot_encoded

# takes a .csv filename
def dataPreprocessing(fileName):
    df = pd.read_csv(fileName, header = None)
    
    #prepare the imput data
    xString = df.iloc[:,:43].to_string(header=False, index=False, index_names = False).split('\n')
    xList = [','.join(ele.split()) for ele in xString] # gives comma separated strings for each row of DataFrame
    xData = []
    for string in xList:
        stringLower = string.lower()
        oneHot = customOneHotEncoder(stringLower)
        xData.append(oneHot)
    xMid = np.array(xData)
    xArray = xMid.transpose(0,2,1) # convert xMid's dim (size, 200, 39) to (size, 39, 200)
    
    #prepare the label data
    df[43] = np.where(pd.isnull(df[47]), 'Normal', 'attack') # replacing anything except 'normal' with 'attack'
    Ydf = df[43]
    #labelName = Ydf.unique().tolist().sort() # sorted 38 label names
    #yArray = Ydf.str.get_dummies().to_numpy() # ndarray of shape(rows/lines, 38)
    yArray = Ydf.to_numpy()
    
    assert xArray.shape[0] == yArray.shape[0], 'unequal input and label sample size'
    
    
    return xArray, yArray # return processed array of input and label

In [60]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

class UNSWDataset(Dataset):
    def __init__(self, fileName):
        self.data = pd.read_csv(fileName, header = None)
        #word embedding here
        idx= [0,1,9]
        for i in idx:
            self.data[i] = self.data[i].astype('category')
            self.data[i] = self.data[i].cat.codes
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # prepare x data
        #string = list(','.join('%s' %x for x in y) for y in self.data.iloc[[idx], :47].values)
        #stringLower = string[0].lower()
        #xData = customOneHotEncoder(stringLower) # Dim (200, 39)
        #xMid = np.array(xData)
        #xArray = xMid.transpose() # should be now (39, 200)
        xArray = self.data.iloc[[idx],:-1].to_numpy()
        
        # prepate y data
        #self.data.iloc[idx, 41] = np.where(self.data.iloc[idx, 41]=='normal', 0, 1) # replacing normals with 0 and anything else with 1
        #yArray = np.where(pd.isnull(self.data.iloc[idx, 47]), 0, 1)
        yArray = self.data.iloc[idx,-1]
        #yArray = Ydf.to_numpy()
    
        #assert xArray.shape == yArray.shape, 'unequal input and label sample size'
        
        return torch.from_numpy(xArray), yArray # returns torch tensor of x and y

In [61]:
params = {'batch_size': 64}
fileNameTrain = 'UNSW-NB15_1_reduced-16-binary-class-redcol.csv'
fileNameTest = 'UNSW-NB15_1_reduced-16-binary-class-redcol.csv'
datasetTrain = UNSWDataset(fileNameTrain)
datasetTest = UNSWDataset(fileNameTest)
dataGeneratorTrain = DataLoader(datasetTrain, None)
dataGeneratorTest = DataLoader(datasetTest, None)

#define sequence



In [62]:
for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
    print (batch_idx, data, target)

0 tensor([[1.1000e+01, 1.0000e+00, 1.0550e-03, 1.3200e+02, 1.6400e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 5.0047e+05, 6.2180e+05,
         2.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.6000e+01, 8.2000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.7000e-02, 1.3000e-02, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.0000e+00, 7.0000e+00,
         1.0000e+00, 3.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00]],
       dtype=torch.float64) tensor(0)
1 tensor([[1.1000e+01, 1.0000e+00, 3.6133e-02, 5.2800e+02, 3.0400e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.7676e+04, 5.0480e+04,
         4.0000e+00, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.3200e+02, 7.6000e+01, 0.0000e+00, 0.0000e+00, 9.8910e+00, 1.0683e+01,
         7.0050e+00, 7.5643e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+

68 tensor([[1.1000e+01, 1.0000e+00, 1.0440e-03, 1.3000e+02, 1.6200e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 4.9808e+05, 6.2069e+05,
         2.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.5000e+01, 8.1000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1000e-02, 8.0000e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 6.0000e+00, 9.0000e+00,
         2.0000e+00, 3.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00]],
       dtype=torch.float64) tensor(0)
69 tensor([[1.1000e+01, 1.0000e+00, 4.5650e-03, 5.6800e+02, 3.0400e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 7.4655e+05, 3.9956e+05,
         4.0000e+00, 4.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.4200e+02, 7.6000e+01, 0.0000e+00, 0.0000e+00, 1.7223e+00, 1.4321e+00,
         1.2253e+00, 1.0207e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000

133 tensor([[1.1000e+01, 1.0000e+00, 1.1060e-03, 1.3200e+02, 1.6400e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 4.7740e+05, 5.9313e+05,
         2.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.6000e+01, 8.2000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1000e-02, 8.0000e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0000e+00, 8.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00]],
       dtype=torch.float64) tensor(0)
134 tensor([[1.1000e+01, 1.0000e+00, 1.1040e-03, 1.3000e+02, 1.6200e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 4.7101e+05, 5.8696e+05,
         2.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.5000e+01, 8.1000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.1000e-02, 9.0000e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

198 tensor([[1.0000e+01, 3.0000e+00, 2.6827e-01, 8.7800e+02, 1.0020e+03, 6.2000e+01,
         2.5200e+02, 2.0000e+00, 2.0000e+00, 5.0000e+00, 2.3588e+04, 2.6153e+04,
         1.0000e+01, 8.0000e+00, 2.5500e+02, 2.5500e+02, 1.6571e+09, 2.7301e+09,
         8.8000e+01, 1.2500e+02, 1.0000e+00, 7.8000e+01, 1.4670e+03, 5.3908e+01,
         2.9808e+01, 3.7051e+01, 5.5114e-02, 8.2770e-03, 4.6837e-02, 0.0000e+00,
         1.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 3.0000e+00, 1.0000e+00,
         1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00]],
       dtype=torch.float64) tensor(1)
199 tensor([[1.1000e+01, 1.0000e+00, 1.1160e-03, 1.3200e+02, 1.6400e+02, 3.1000e+01,
         2.9000e+01, 0.0000e+00, 0.0000e+00, 2.0000e+00, 4.7312e+05, 5.8781e+05,
         2.0000e+00, 2.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         6.6000e+01, 8.2000e+01, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         1.2000e-02, 8.0000e-03, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

257 tensor([[1.0000e+01, 3.0000e+00, 6.0162e-02, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 1.9201e+05, 2.0651e+05,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 4.1989e+09, 2.2382e+09,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 2.4956e+02, 8.1458e+00,
         3.9349e+00, 3.4676e+00, 1.3530e-03, 1.2070e-03, 1.4600e-04, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00, 2.0000e+00,
         9.0000e+00, 4.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00]],
       dtype=torch.float64) tensor(0)
258 tensor([[1.0000e+01, 3.0000e+00, 1.4197e+01, 1.0520e+03, 8.2400e+02, 3.1000e+01,
         2.9000e+01, 2.0000e+00, 2.0000e+00, 5.0000e+00, 5.1898e+02, 4.1811e+02,
         8.0000e+00, 1.0000e+01, 2.5500e+02, 2.5500e+02, 1.0849e+08, 2.2592e+09,
         1.3200e+02, 8.2000e+01, 0.0000e+00, 0.0000e+00, 1.6901e+05, 4.4537e+03,
         2.0281e+03, 1.5774e+03, 6.9100e-04, 5.5100e-04, 1.4000e-04, 0.00

...

       dtype=torch.float64) tensor(0)
862 tensor([[1.0000e+01, 3.0000e+00, 2.8696e+01, 4.5520e+03, 2.9760e+03, 3.1000e+01,
         2.9000e+01, 7.0000e+00, 7.0000e+00, 0.0000e+00, 1.2202e+03, 8.0011e+02,
         2.6000e+01, 2.8000e+01, 2.5500e+02, 2.5500e+02, 2.6405e+09, 2.6398e+09,
         1.7500e+02, 1.0600e+02, 0.0000e+00, 0.0000e+00, 1.5369e+05, 4.5587e+03,
         1.1478e+03, 1.0628e+03, 7.3900e-04, 5.7700e-04, 1.6200e-04, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.2000e+01, 8.0000e+00,
         7.0000e+00, 1.1000e+01, 1.0000e+00, 1.0000e+00, 1.0000e+00]],
       dtype=torch.float64) tensor(0)
863 tensor([[1.0000e+01, 3.0000e+00, 2.8513e+01, 4.6560e+03, 2.9760e+03, 3.1000e+01,
         2.9000e+01, 7.0000e+00, 7.0000e+00, 0.0000e+00, 1.2598e+03, 8.0525e+02,
         2.8000e+01, 2.8000e+01, 2.5500e+02, 2.5500e+02, 3.0605e+09, 3.0852e+09,
         1.6600e+02, 1.0600e+02, 0.0000e+00, 0.0000e+00, 1.4354e+05, 4.4215e+03,
         1.0560e+03, 1.0560e+03, 7.

       dtype=torch.float64) tensor(0)
914 tensor([[1.0000e+01, 3.0000e+00, 3.6995e+00, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 3.1226e+03, 3.3583e+03,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 7.2102e+08, 2.7222e+09,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 1.5773e+04, 4.4796e+02,
         2.4661e+02, 2.0693e+02, 1.8354e-01, 1.8168e-01, 1.8590e-03, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.7000e+01, 1.2000e+01,
         1.2000e+01, 1.5000e+01, 1.4000e+01, 1.0000e+00, 2.0000e+00]],
       dtype=torch.float64) tensor(0)
915 tensor([[1.0000e+01, 3.0000e+00, 4.3588e+00, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 2.6503e+03, 2.8503e+03,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 1.9095e+08, 3.4277e+08,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 2.0695e+04, 4.4463e+02,
         2.9056e+02, 2.5135e+02, 8.

       dtype=torch.float64) tensor(0)
966 tensor([[1.0000e+01, 3.0000e+00, 5.1532e+00, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 2.2417e+03, 2.4109e+03,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 4.4347e+08, 2.2701e+09,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 2.3511e+04, 4.9737e+02,
         3.4349e+02, 3.0306e+02, 1.2810e-03, 1.1260e-03, 1.5500e-04, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.3000e+01, 1.3000e+01,
         1.1000e+01, 1.2000e+01, 1.1000e+01, 1.0000e+00, 7.0000e+00]],
       dtype=torch.float64) tensor(0)
967 tensor([[1.0000e+01, 3.0000e+00, 4.2430e+00, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 2.7226e+03, 2.9281e+03,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 5.0605e+08, 2.2781e+09,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 1.7589e+04, 5.8266e+02,
         2.8284e+02, 2.4952e+02, 1.

1019 tensor([[1.0000e+01, 3.0000e+00, 5.7560e+00, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 2.0069e+03, 2.1584e+03,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 5.0347e+08, 2.5037e+08,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 2.8905e+04, 6.4515e+02,
         3.8371e+02, 3.3645e+02, 3.7199e-02, 3.6375e-02, 8.2400e-04, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.3000e+01, 9.0000e+00,
         1.0000e+01, 1.5000e+01, 1.0000e+01, 2.0000e+00, 6.0000e+00]],
       dtype=torch.float64) tensor(0)
1020 tensor([[1.0000e+01, 3.0000e+00, 8.0902e-01, 1.5400e+03, 1.6440e+03, 3.1000e+01,
         2.9000e+01, 4.0000e+00, 4.0000e+00, 0.0000e+00, 1.4279e+04, 1.5357e+04,
         1.6000e+01, 1.8000e+01, 2.5500e+02, 2.5500e+02, 3.1607e+09, 1.0725e+09,
         9.6000e+01, 9.1000e+01, 0.0000e+00, 0.0000e+00, 3.3656e+03, 9.9839e+01,
         5.3911e+01, 4.7563e+01, 6.0000e-04, 4.4900e-04, 1.5100e-04, 0.

KeyboardInterrupt: 

In [4]:
params = {'batch_size': 64, 'shuffle': True}
filedataset = 'UNSW-NB15_1.csv'
dataset = NSLKDDDataset(filedataset)
total_length = dataset.__len__()
train_length = int(0.6 * total_length)
test_length = int(0.2 * total_length)
val_length = total_length - train_length - test_length
datasetTrain, datasetVal, datasetTest = torch.utils.data.random_split(dataset, [train_length, val_length, test_length])
dataGeneratorTrain = DataLoader(datasetTrain, **params)
dataGeneratorTest = DataLoader(datasetTest, **params)
dataGeneratorVal = DataLoader(datasetVal, **params)

C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (1,3,47) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [7]:
datasetTrain.__len__()

420000

In [7]:
root = './data/mnist'
batch_size = 64
n_classes = 2
input_channels = 42
seq_length = int(200)
epochs = 10
steps = 0

In [8]:
#train_loader, test_loader = data_generator(root, batch_size)

permute = torch.Tensor(np.random.permutation(784).astype(np.float64)).long()
channel_sizes = [32] * 6 #hidden nodes times levels 
kernel_size = 5
nheads = 1

model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=0.25)
#input_size, output_size, num_channels, 
#kernel_size, dropout, num_sub_blocks, temp_attn, nheads, en_res,                              num_sub_blocks
#conv, key_size
#
# nheads, key_size


lr = 1e-5
optimizer = getattr(optim, 'Adam')(model.parameters(), lr=lr)

In [9]:
def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(dataGeneratorTrain):
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        optimizer.zero_grad()
        #print(data.shape)
        data = data.view(-1, input_channels, seq_length)
        data, target = Variable(data), Variable(target)
        #print('data Shape: {} target shape: {} data type: {}'.format(data.shape, target.shape, type(data)))
        #print(target)
        optimizer.zero_grad()
        #print(data[0])
        data = data.type(torch.FloatTensor)
        output = model(data)
        #print(output.shape)
        target = target.type(torch.LongTensor)
        loss1 = torch.nn.CrossEntropyLoss()
        loss = F.nll_loss(output, target) # negative log likelihood
        loss = loss1(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss
        steps += seq_length
        #print(batch_idx)
        if batch_idx > 0 and batch_idx % 800 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(dataGeneratorTrain.dataset),
                100. * batch_idx / len(dataGeneratorTrain), train_loss.item()/100, steps))
            train_loss = 0

In [10]:
def test():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorTest:
            model.eval()
            #print('checkpoint 1')
            #data = data.view(-1, input_channels, seq_length)
            #print('checkpoint 2')
            data = data.type(torch.FloatTensor)
            #print('checkpoint 3')
            target = target.type(torch.LongTensor)
            #print('checkpoint 4')
            data, target = Variable(data, volatile=True), Variable(target)
            #print('checkpoint 5')
            output = model(data)
            #print('checkpoint 6')
            #loss1 = torch.nn.CrossEntropyLoss()
            test_loss += F.nll_loss(output, target, size_average=False).item()
            #print('checkpoint 7')
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            #print('checkpoint 8')
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
            #print('checkpoint 9')

        test_loss /= len(dataGeneratorTest.dataset)
        #print('checkpoint 10')
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorTest.dataset),
            100. * correct / len(dataGeneratorTest.dataset)))
        return test_loss

In [11]:
if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Train Epoch: 1 [51200/420000 (12%)]	Loss: 2.416590	Steps: 160200
Train Epoch: 1 [102400/420000 (24%)]	Loss: 1.124084	Steps: 320200
Train Epoch: 1 [153600/420000 (37%)]	Loss: 1.088910	Steps: 480200
Train Epoch: 1 [204800/420000 (49%)]	Loss: 1.078178	Steps: 640200
Train Epoch: 1 [256000/420000 (61%)]	Loss: 1.063734	Steps: 800200
Train Epoch: 1 [307200/420000 (73%)]	Loss: 0.998640	Steps: 960200
Train Epoch: 1 [358400/420000 (85%)]	Loss: 0.944188	Steps: 1120200
Train Epoch: 1 [409600/420000 (98%)]	Loss: 0.834705	Steps: 1280200


C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\lromo\OneDrive\Documents\virtualenvs\netsec\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.0814, Accuracy: 135506/140000 (97%)

Train Epoch: 2 [51200/420000 (12%)]	Loss: 0.553623	Steps: 1472800
Train Epoch: 2 [102400/420000 (24%)]	Loss: 0.314074	Steps: 1632800
Train Epoch: 2 [153600/420000 (37%)]	Loss: 0.211456	Steps: 1792800
Train Epoch: 2 [204800/420000 (49%)]	Loss: 0.170805	Steps: 1952800
Train Epoch: 2 [256000/420000 (61%)]	Loss: 0.168684	Steps: 2112800
Train Epoch: 2 [307200/420000 (73%)]	Loss: 0.164602	Steps: 2272800
Train Epoch: 2 [358400/420000 (85%)]	Loss: 0.154673	Steps: 2432800
Train Epoch: 2 [409600/420000 (98%)]	Loss: 0.164506	Steps: 2592800

Test set: Average loss: 0.0183, Accuracy: 139021/140000 (99%)

Train Epoch: 3 [51200/420000 (12%)]	Loss: 0.152688	Steps: 2785400
Train Epoch: 3 [102400/420000 (24%)]	Loss: 0.146438	Steps: 2945400
Train Epoch: 3 [153600/420000 (37%)]	Loss: 0.155771	Steps: 3105400
Train Epoch: 3 [204800/420000 (49%)]	Loss: 0.157154	Steps: 3265400
Train Epoch: 3 [256000/420000 (61%)]	Loss: 0.143733	Steps: 3425400
Tra

KeyboardInterrupt: 

In [12]:
#NEED TO DO THIS ONE
from sklearn.metrics import classification_report
def get_metrics(dataLoader):
    total = 0
    correct = 0
    precision = 0
    recall = 0
    f1_score = 0
    accuracy = 0
    with torch.no_grad():
        for data, target in dataLoader: # just 1 batch
            model.eval()
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            #print(predicted)
            total+=target.size(0)
            correct+=(predicted == target).sum().item()
            report = classification_report(target, predicted, output_dict=True)
            precision += report['macro avg']['precision']
            recall += report['macro avg']['recall']
            f1_score += report['macro avg']['f1-score']
            accuracy += report['accuracy']
            print(report)
    print("Precision: {}, Recall: {}, F1-Score: {}, Accuracy: {}, AccuracyCust: {}".format(precision, recall, f1_score, accuracy, correct/total))
    

In [ ]:
#for now don't do thisone
#fileNameVal = 'UNSW_NB15_testing-set.csv'
#datasetVal = NSLKDDDataset(fileNameVal)
params = {'batch_size': 22544, 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
#RuntimeError: expected scalar type Int but found Float
#get_metrics(dataGeneratorVal)
def val():
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in dataGeneratorVal:
            model.eval()
            #data = data.view(-1, input_channels, seq_length)
            data = data.type(torch.FloatTensor)
            target = target.type(torch.LongTensor)
            data, target = Variable(data, volatile=True), Variable(target)
            output = model(data)
            #loss1 = torch.nn.CrossEntropyLoss()
            #test_loss += F.nll_loss(output, target, size_average=False).item()
            #test_loss += loss1(output, target).item()
            #print(output.data.max(1, keepdim=True)[1])
            pred = output.data.max(1, keepdim=True)[1]
            print(pred)
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(dataGeneratorTest.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(dataGeneratorVal.dataset),
            100. * correct / len(dataGeneratorVal.dataset)))
        return test_loss

val()


In [13]:
#do this one last
params = {'batch_size': datasetVal.__len__(), 'shuffle': True}
dataGeneratorVal = DataLoader(datasetVal, **params)
get_metrics(dataGeneratorVal)

{'0': {'precision': 0.999888706697631, 'recall': 0.9937102280687524, 'f1-score': 0.9967898933416175, 'support': 135617}, '1': {'precision': 0.8366526235158943, 'recall': 0.9965784671532847, 'f1-score': 0.9096398084530501, 'support': 4384}, 'accuracy': 0.9938000442853979, 'macro avg': {'precision': 0.9182706651067627, 'recall': 0.9951443476110186, 'f1-score': 0.9532148508973338, 'support': 140001}, 'weighted avg': {'precision': 0.9947771218613174, 'recall': 0.9938000442853979, 'f1-score': 0.9940608701764154, 'support': 140001}}
Precision: 0.9182706651067627, Recall: 0.9951443476110186, F1-Score: 0.9532148508973338, Accuracy: 0.9938000442853979, AccuracyCust: 0.9938000442853979


In [14]:
#do this one
params = {'batch_size': datasetTest.__len__(), 'shuffle': True}
dataGeneratorTest = DataLoader(datasetTest, **params)
get_metrics(dataGeneratorTest)

{'0': {'precision': 0.9998960607005509, 'recall': 0.9939043289596032, 'f1-score': 0.9968911917098446, 'support': 135506}, '1': {'precision': 0.8443271767810027, 'recall': 0.9968847352024922, 'f1-score': 0.9142857142857144, 'support': 4494}, 'accuracy': 0.994, 'macro avg': {'precision': 0.9221116187407767, 'recall': 0.9953945320810478, 'f1-score': 0.9555884529977795, 'support': 140000}, 'weighted avg': {'precision': 0.9949022995267334, 'recall': 0.994, 'f1-score': 0.99423955588453, 'support': 140000}}
Precision: 0.9221116187407767, Recall: 0.9953945320810478, F1-Score: 0.9555884529977795, Accuracy: 0.994, AccuracyCust: 0.994


In [17]:
torch.save(model, "tcn_unsw.pth")

In [34]:
import pandas as pd

data = pd.read_csv("UNSW-NB15_1_reduced-16-binary-class-redcol.csv", header=None)
data.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
count,40762,40762,40762.000000,4.076200e+04,4.076200e+04,40762.000000,40762.000000,40762.000000,40762.000000,40762,...,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000,40762.000000
unique,13,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,tcp,FIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,29688,29153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,1.114260,5.954914e+03,4.047243e+04,54.699818,47.195918,6.153157,18.010402,NaN,...,0.016044,0.017001,5.043717,4.739095,3.542245,4.039399,1.606619,1.046906,1.909327,0.079314
std,NaN,NaN,5.039567,8.336349e+04,1.862701e+05,68.360785,62.596066,32.761024,66.327121,NaN,...,0.125648,0.148529,4.012933,3.738378,2.614704,2.996480,1.672633,0.307345,1.489764,0.270232
min,NaN,NaN,0.000000,4.600000e+01,0.000000e+00,0.000000,0.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
25%,NaN,NaN,0.004464,5.200000e+02,3.040000e+02,31.000000,29.000000,0.000000,0.000000,NaN,...,0.000000,0.000000,2.000000,2.000000,2.000000,2.000000,1.000000,1.000000,1.000000,0.000000
50%,NaN,NaN,0.036865,1.540000e+03,1.956000e+03,31.000000,29.000000,4.000000,4.000000,NaN,...,0.000000,0.000000,4.000000,4.000000,3.000000,3.000000,1.000000,1.000000,1.000000,0.000000
75%,NaN,NaN,0.528106,3.806000e+03,1.016800e+04,31.000000,29.000000,7.000000,11.000000,NaN,...,0.000000,0.000000,7.000000,7.000000,5.000000,5.000000,1.000000,1.000000,2.000000,0.000000


In [20]:
for i in range(41):
    print(data[i].isnull().values.any())

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40762 entries, 0 to 40761
Data columns (total 42 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       40762 non-null  int8   
 1   1       40762 non-null  int8   
 2   2       40762 non-null  float64
 3   3       40762 non-null  int64  
 4   4       40762 non-null  int64  
 5   5       40762 non-null  int64  
 6   6       40762 non-null  int64  
 7   7       40762 non-null  int64  
 8   8       40762 non-null  int64  
 9   9       40762 non-null  int8   
 10  10      40762 non-null  float64
 11  11      40762 non-null  float64
 12  12      40762 non-null  int64  
 13  13      40762 non-null  int64  
 14  14      40762 non-null  int64  
 15  15      40762 non-null  int64  
 16  16      40762 non-null  int64  
 17  17      40762 non-null  int64  
 18  18      40762 non-null  int64  
 19  19      40762 non-null  int64  
 20  20      40762 non-null  int64  
 21  21      40762 non-null  int64  
 22

In [27]:
data[0] = data[0].astype('category')
data[0].cat.codes


0        11
1        11
2        11
3        11
4        11
         ..
40757    10
40758    10
40759    10
40760    10
40761    11
Length: 40762, dtype: int8

In [28]:
data[0].unique().shape

(13,)

In [29]:
data[1] = data[1].astype('category')
data[1].cat.codes
data[1].unique().shape

(9,)

In [30]:
data[9] = data[9].astype('category')
data[9].cat.codes
data[9].unique().shape

(13,)

In [36]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,11,1,0.001055,132,164,31,29,0,0,2,...,0,0,3,7,1,3,1,1,1,0
1,11,1,0.036133,528,304,31,29,0,0,0,...,0,0,2,4,2,3,1,1,2,0
2,11,1,0.001119,146,178,31,29,0,0,2,...,0,0,12,8,1,2,2,1,1,0
3,11,1,0.001209,132,164,31,29,0,0,2,...,0,0,6,9,1,1,1,1,1,0
4,11,1,0.001169,146,178,31,29,0,0,2,...,0,0,7,9,1,1,1,1,1,0


In [35]:
idx= [0,1,9]
for i in idx:
    data[i] = data[i].astype('category')
    data[i] = data[i].cat.codes